In [1]:
import hashlib
import socket
import time
import threading
import string    
import random

In [2]:
startString = "Start_Connection"
privateString = "PRIVATE_KEY"

startHex = hashlib.sha1(startString.encode()).hexdigest()
privateHex = hashlib.sha1(privateString.encode()).hexdigest()
print(startHex, privateHex)

f52f1671ca7c30bb346e893842e022418c9f41eb e09718150b93ebc2f4e33cba41ff13ec23cd9bd5


In [3]:
hashlib.sha1(startString.encode() + privateString.encode()).hexdigest()

'0135e1b461baef3a589aab10e33421d7dde832ba'

In [4]:
hashlib.sha1((startString + privateString).encode()).hexdigest()

'0135e1b461baef3a589aab10e33421d7dde832ba'

In [15]:
class ClientSocket:
    def __init__(self, HOST, PORT):
        self.HOST = HOST
        self.PORT = PORT
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        

    def start(self):
        self.socket.connect((self.HOST, self.PORT))
        print(f'[CONNECTING] {self.HOST}:{self.PORT}')

        thread = threading.Thread(target=self.handle_connection)
        thread.start()
        print(f'[ACTIVE THREADS] {threading.activeCount()-1}')


    def handle_connection(self):
        if self.auth():
            connected = True
            while connected:
                pass

        self.socket.send("DISCONNECT".encode())
        print(f'[DISCONNECTING]')


    def auth(self):
        startString = "Start_Connection"
        privateString = "PRIVATE_KEY_THIRTY_TWO_CHARACTER"

        self.socket.send(startString.encode())
        print(f'[AUTH SENT] Start: {startString}')

        randomString = self.socket.recv(1024).decode()
        print(f'[AUTH RECIEVED] Random: {randomString}')

        sha1Result = hashlib.sha1(privateString.encode() + randomString.encode()).hexdigest()
        self.socket.send(sha1Result.encode())
        print(f'[AUTH SENT] SHA1: {sha1Result}')

        message = self.socket.recv(1024).decode()
        print(f'[AUTH RECIEVED] Message: {message}')

        if message == "Authentication unsuccesful.":
            return False

        global answer 
        answer = input()
        self.socket.send(answer.encode())
        print(f'[AUTH SENT] Answer: {answer}')

        return True


    def sendPacket(self, packet_type):
        payload = '---'
        packet_size = 0

        if packet_type == 3: #Guess
            payload = bytes(str(input("Guess number between [0-36]: ")).encode())
            packet_size = len(payload)

        header = bytes([packet_type, packet_size])
  
        self.socket.send(header)
        self.socket.send(payload)
        print(f'[SENT] Header: {header} - Payload: {payload}')
    

In [ ]:
if __name__=="__main__":
    c = ClientSocket('localhost', 3001)
    #c.send(bytes([1, 36]))
    c.start()

In [10]:
packet_type = 3
payload = input("What is your guess? Number, even, odd?").encode()
payload_size = len(payload)

packet = bytes([packet_type, payload_size])+payload

print(packet, packet[0], packet[1], packet[2:2+packet[1]])


b'\x03\x03345' 3 3 b'345'


In [12]:
a = (258).to_bytes(2, byteorder="big", signed=True)
print(len(a), a)

2 b'\x01\x02'
